In [ ]:
import pandas as pd
import csv
import re
from google.colab import drive
import logging
import os

In [ ]:
!pip install peft
!pip install -U transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Projects/backend/models



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Projects/backend/models


In [ ]:

df = pd.read_csv('sufinama_ghazals.csv')
df1 = pd.read_csv('rekhta.csv')

In [ ]:
df = pd.concat([df1, df], ignore_index=True, sort=False)

In [ ]:
df_clean = df.dropna(subset=["ghazal_en", "ghazal_ur"])

In [ ]:
df_clean

,ghazal_url,name,all_words,ghazal_en,ghazal_ur
0,https://www.rekhta.org/ghazals/intizaar-e-diid...,gawwas-qureshi,"[{'id': '\\0brr', 'text': 'intizār-e-dīd'}, {'...","intizār-e-dīd meñ yuuñ aañkh pathrā.ī ki bas ,...","انتظار دید میں یوں آنکھ پتھرائی کہ بس ,مرتے مر..."
1,https://www.rekhta.org/ghazals/har-mausam-e-ta...,gauhar-shaikh-purwi,"[{'id': '\\1gvt', 'text': 'har'}, {'id': '6qwo...","har mausam-e-tapāñ meñ gul-e-tar luTā.iye ,jab...","ہر موسم تپاں میں گل تر لٹائیے ,جب لوگ تشنہ لب ..."
2,https://www.rekhta.org/ghazals/ik-sadaaqat-hai...,ghayur-jafari,"[{'id': '\\1m1h', 'text': 'ik'}, {'id': '\\1cl...","ik sadāqat hai be-rag-o-be-pai ,terī muTThī me...","اک صداقت ہے بے رگ و بے پے ,تیری مٹھی میں بند ہ..."
3,https://www.rekhta.org/ghazals/aks-kii-kahaani...,ghalib-irfan,"[{'id': '\\2197', 'text': 'aks'}, {'id': '\\1x...","aks kī kahānī kā iqtibās ham hī the ,aa.ine ke...","عکس کی کہانی کا اقتباس ہم ہی تھے ,آئنے کے باہر..."
4,https://www.rekhta.org/ghazals/saanson-kii-jal...,ganesh-bihari-tarz,"[{'id': '\\1buy', 'text': 'sāñsoñ'}, {'id': '\...",sāñsoñ kī jal-tarañg par naġhma-e-ishq gaa.e j...,"سانسوں کی جل ترنگ پر نغمۂ عشق گائے جا ,اے مری ..."
...,...,...,...,...,...
69300,https://sufinama.org/ghazals/mahshar-badayuni-...,mahshar-badayuni,"[{'id': '\\0rl2', 'text': 'pā-ba-jaulāñ'}, {'i...",pā-ba-jaulāñ 'arsa-e-sarv-o-sabā meñ aa.e haiñ...,"پا بجولاں عرصۂ سرو و صبا میں آئے ہیں ,ہم بھی ک..."
69304,https://sufinama.org/ghazals/taha-aziz-ghazals-3,taha-aziz,"[{'id': 'zalI4FSrSF2xU1TxhP7Nmg==', 'text': 'क...","क्या करेंगे होके फिर आज़ाद हम ,ख़ुश हैं तेरी क़...","क्या करेंगे होके फिर आज़ाद हम ,ख़ुश हैं तेरी क़..."
69306,https://sufinama.org/ghazals/shah-qateel-danap...,qateel-danapuri,"[{'id': 'nHqR6jVfYxKSwY9gyo7xNg==', 'text': 'म...",मकीं हो जाओ तुम जिस में वो दिल वीराँ नहीं होता...,मकीं हो जाओ तुम जिस में वो दिल वीराँ नहीं होता...
69307,https://sufinama.org/ghazals/mahmood-ali-khan-...,mahmood-ali-saba,"[{'id': '\\08bk', 'text': 'vahīñ'}, {'id': '\\...",vahīñ par jaake Thahrā ahl-e-dil ahl-e-dil kā ...,وہیں پر جاکے ٹھہرا اہل دل اہل دل کا کارواں پہل...


In [ ]:
# --- Step 1: Detect Inconsistencies ---
def has_hindi(text):
    """Check for Devanagari (Hindi) characters in Urdu column <button class="citation-flag" data-index="1">"""
    return any('\u0900' <= char <= '\u097F' for char in str(text))

def invalid_roman(text):
    """Check if Roman Urdu contains Urdu script <button class="citation-flag" data-index="5">"""
    if pd.isna(text):
        return True
    return any('\u0600' <= char <= '\u06FF' for char in str(text))

# Flag script inconsistencies and missing values <button class="citation-flag" data-index="5"><button class="citation-flag" data-index="6">
df["has_hindi"] = df["ghazal_ur"].apply(has_hindi)
df["invalid_roman"] = df["ghazal_en"].apply(invalid_roman)
df["missing_ghazal_ur"] = df["ghazal_ur"].isna()  # Flag original NaNs <button class="citation-flag" data-index="5">
df["missing_ghazal_en"] = df["ghazal_en"].isna()  # Flag original NaNs <button class="citation-flag" data-index="6">

# Combine all flags to define inconsistent data
inconsistent_data = df[
    df["has_hindi"] |
    df["invalid_roman"] |
    df["missing_ghazal_ur"] |
    df["missing_ghazal_en"]
]

# Clean data (rows without inconsistencies or missing values)
clean_data = df[
    ~(
        df["has_hindi"] |
        df["invalid_roman"] |
        df["missing_ghazal_ur"] |
        df["missing_ghazal_en"]
    )
]


In [ ]:
clean_data

,ghazal_url,name,all_words,ghazal_en,ghazal_ur,has_hindi,invalid_roman,missing_ghazal_ur,missing_ghazal_en
0,https://www.rekhta.org/ghazals/intizaar-e-diid...,gawwas-qureshi,"[{'id': '\\0brr', 'text': 'intizār-e-dīd'}, {'...","intizār-e-dīd meñ yuuñ aañkh pathrā.ī ki bas ,...","انتظار دید میں یوں آنکھ پتھرائی کہ بس ,مرتے مر...",False,False,False,False
1,https://www.rekhta.org/ghazals/har-mausam-e-ta...,gauhar-shaikh-purwi,"[{'id': '\\1gvt', 'text': 'har'}, {'id': '6qwo...","har mausam-e-tapāñ meñ gul-e-tar luTā.iye ,jab...","ہر موسم تپاں میں گل تر لٹائیے ,جب لوگ تشنہ لب ...",False,False,False,False
2,https://www.rekhta.org/ghazals/ik-sadaaqat-hai...,ghayur-jafari,"[{'id': '\\1m1h', 'text': 'ik'}, {'id': '\\1cl...","ik sadāqat hai be-rag-o-be-pai ,terī muTThī me...","اک صداقت ہے بے رگ و بے پے ,تیری مٹھی میں بند ہ...",False,False,False,False
3,https://www.rekhta.org/ghazals/aks-kii-kahaani...,ghalib-irfan,"[{'id': '\\2197', 'text': 'aks'}, {'id': '\\1x...","aks kī kahānī kā iqtibās ham hī the ,aa.ine ke...","عکس کی کہانی کا اقتباس ہم ہی تھے ,آئنے کے باہر...",False,False,False,False
4,https://www.rekhta.org/ghazals/saanson-kii-jal...,ganesh-bihari-tarz,"[{'id': '\\1buy', 'text': 'sāñsoñ'}, {'id': '\...",sāñsoñ kī jal-tarañg par naġhma-e-ishq gaa.e j...,"سانسوں کی جل ترنگ پر نغمۂ عشق گائے جا ,اے مری ...",False,False,False,False
...,...,...,...,...,...,...,...,...,...
69292,https://sufinama.org/ghazals/gar-raah-e-talab-...,mahboob-warsi-gayavi,"[{'id': '\\1wx7', 'text': 'gar'}, {'id': '\\1q...",gar rāh-e-talab meñ sādiq hai shikva bhī dil-e...,گر راہ طلب میں صادق ہے شکوہ بھی دل ناشاد نہ کر...,False,False,False,False
69293,https://sufinama.org/ghazals/mahmood-ahmad-rab...,mahmood-ahmad-rabbani,"[{'id': '\\4xs5', 'text': 'jamāl-e-nūr-e-rahmā...",jamāl-e-nūr-e-rahmāñ par ġhazal kahnā 'ibādat ...,"جمال نور رحماں پر غزل کہنا عبادت ہے ,وہ خلقت ک...",False,False,False,False
69296,https://sufinama.org/ghazals/mahmood-alam-ghazals,mahmood-alam,"[{'id': '\\11ez', 'text': 'lab-e-ḳhāmosh'}, {'...","lab-e-ḳhāmosh kā bayāñ huuñ maiñ ,kanz-e-maḳhf...","لب خاموش کا بیاں ہوں میں ,کنز مخفی ہے وہ عیاں ...",False,False,False,False
69300,https://sufinama.org/ghazals/mahshar-badayuni-...,mahshar-badayuni,"[{'id': '\\0rl2', 'text': 'pā-ba-jaulāñ'}, {'i...",pā-ba-jaulāñ 'arsa-e-sarv-o-sabā meñ aa.e haiñ...,"پا بجولاں عرصۂ سرو و صبا میں آئے ہیں ,ہم بھی ک...",False,False,False,False


In [ ]:

# Save results <button class="citation-flag" data-index="6">
inconsistent_data.to_csv("inconsistent_data.csv", index=False)
clean_data.to_csv("clean_ghazals.csv", index=False)

In [ ]:
# df.loc[2294,"ghazal_url"]

In [ ]:
df.loc[df["name"] == 'baba-shah-hussaini']

,ghazal_url,name,all_words,ghazal_en,ghazal_ur


In [ ]:
# Load data
df = pd.read_csv("clean_ghazals.csv")
# Create training examples
training_data = []

In [ ]:
df["ghazal_ur"] = df["ghazal_ur"].fillna("[MISSING]")
df["ghazal_en"] = df["ghazal_en"].fillna("[MISSING]")

In [ ]:
df.loc[df['ghazal_ur'] == "[MISSING]"]

,ghazal_url,name,all_words,ghazal_en,ghazal_ur,has_hindi,invalid_roman,missing_ghazal_ur,missing_ghazal_en


In [ ]:
for _, row in df.iterrows():
    poet = row["name"].replace("-", " ").title()

    # Task 1: Urdu → Roman Conversion
    training_data.append({
        "instruction": f"Convert this Urdu ghazal by {poet} to Roman Urdu",
        "input": row["ghazal_ur"],
        "response": row["ghazal_en"]
    })

    # Task 2: Roman → Urdu Conversion
    training_data.append({
        "instruction": f"Convert this Roman Urdu ghazal by {poet} to traditional Urdu script",
        "input": row["ghazal_en"],
        "response": row["ghazal_ur"]
    })

    # Task 3: Poet Attribution
    training_data.append({
        "instruction": "Who wrote this ghazal?",
        "input": row["ghazal_ur"],
        "response": poet
    })

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = pad_sequence([item["input_ids"] for item in batch], batch_first=True)
    attention_mask = pad_sequence([item["attention_mask"] for item in batch], batch_first=True)
    labels = pad_sequence([item["labels"] for item in batch], batch_first=True)
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

In [ ]:
# !pip uninstall -y tensorflow && pip install tensorflow-cpu

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from accelerate import Accelerator
import torch
from torch.utils.data import Dataset

In [ ]:


# Quantization config
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
)

# Load model
model_name = "qwen/Qwen2-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

# Apply LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
)
model = get_peft_model(model, lora_config)
# model.gradient_checkpointing_enable()

class LazyDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        inputs = example["instruction"] + " " + example["input"]
        tokenized = tokenizer(
            text=inputs,
            text_pair=example["response"],
            truncation=True,
            padding="max_length",
            max_length=256,
            return_tensors="pt"
        )
        labels = tokenized["input_ids"].clone()
        labels[labels == tokenizer.pad_token_id] = -100

        return {
            "input_ids": tokenized["input_ids"].squeeze(0),
            "attention_mask": tokenized["attention_mask"].squeeze(0),
            "labels": labels.squeeze(0)  # Include labels
        }


# Prepare data
dataset = LazyDataset(training_data, tokenizer)
train_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    collate_fn=collate_fn  # Add this
)
# Accelerate setup
accelerator = Accelerator()
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-5
)
model, optimizer, train_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
for batch in train_dataloader:
    print("Labels shape:", batch["labels"].shape)  # Should match input_ids shape
    break

Labels shape: torch.Size([2, 256])


In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen-urdu-test",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    fp16=True,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Resume training from the last checkpoint
trainer.train(resume_from_checkpoint=True)

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3423: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you s

Step,Training Loss
410,2.670400
420,2.662000
430,2.662800
440,2.700600
450,2.625900
460,2.599900
470,2.658300
480,2.587900
490,2.604500
500,2.598900


Step,Training Loss
410,2.670400
420,2.662000
430,2.662800
440,2.700600
450,2.625900
460,2.599900
470,2.658300
480,2.587900
490,2.604500
500,2.598900


TrainOutput(global_step=76533, training_loss=1.8446098945413516, metrics={'train_runtime': 74814.5922, 'train_samples_per_second': 8.184, 'train_steps_per_second': 1.023, 'total_flos': 3.370931292955607e+17, 'train_loss': 1.8446098945413516, 'epoch': 2.999970601781532})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load with proper tokenizer settings
tokenizer = AutoTokenizer.from_pretrained(
    "qwen/Qwen2-0.5B",
    pad_token="<|endoftext|>",  # Qwen's EOS token
    padding_side="left"  # Crucial for generation
)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    "qwen/Qwen2-0.5B",
    device_map="auto",
    pad_token_id=tokenizer.pad_token_id  # Match tokenizer
)

# Load LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    "./qwen-urdu-test/checkpoint-76533"  # Your specific checkpoint
)

# Critical configuration for generation
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

def generate_safe(prompt, max_length=200):
    # Tokenize with attention mask
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to(model.device)

    # Generate with proper config
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,  # Pass mask
        max_new_tokens=max_length,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode only new tokens
    input_length = inputs.input_ids.shape[1]
    return tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)

In [ ]:
def test_urdu_features():
    # Test 1: Right-to-Left Consistency
    print("RTL Test:", generate_safe("اردو میں لکھیں: میرا نام علی ہے"))

    # Test 2: Poet Attribution
    print("\nPoet Test:", generate_safe(
        "Who wrote this ghazal? دل سے جو بات نکلتی ہے اثر رکھتی ہے"
    ))

    # Test 3: Mixed Script Handling
    print("\nMixed Script Test:", generate_safe(
        "Convert: میں use کرتا ہوں Roman Urdu کو"
    ))

test_urdu_features()

RTL Test:  یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے یہاں ہے

Poet Test:  ,کہیں ہے کہیں ہے کہیں ہے کہیں ہے کہیں ہے ,کہیں ہے کہیں ہے کہیں ہے کہیں ہے کہیں ہے ,کہیں ہے کہیں ہے کہیں ہے کہیں ہے کہیں ہے ,کہیں ہے کہیں ہے کہیں ہے کہیں ہے کہیں ہے ,کہیں ہے کہیں ہے کہیں ہے کہیں ہے کہیں ہے ,کہیں ہے کہیں ہے کہیں 

Mixed Script Test: ئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کوئی یاد ہے کو


In [ ]:
# Install required libraries
# !pip install transformers torch

# Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the DeepSeek-R1 model and tokenizer
model_name = "deepseek-ai/deepseek-llm-7b-chat"  # Replace with the correct model path if not on Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define an Urdu input prompt
urdu_prompt = "محبت کے بارے میں ایک شعر کہو"  # "Write a poem about love"

# Tokenize the input
inputs = tokenizer(urdu_prompt, return_tensors="pt")

# Generate text
outputs = model.generate(
    inputs.input_ids,
    max_length=100,  # Adjust the length of the generated text
    num_return_sequences=1,  # Number of responses to generate
    no_repeat_ngram_size=2,  # Avoid repeating phrases
    top_k=50,  # Sampling parameter
    top_p=0.95,  # Nucleus sampling parameter
    temperature=0.7,  # Controls randomness
)

# Decode and print the generated Urdu text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Urdu Text:")
print(generated_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Urdu Text:
محبت کے بارے میں ایک شعر کہوچکیوٹیٰ - حب کی گنجازۍ اور دوستی کا بھارت عربستان اپنۡ زِرۃ بالا رکڻ ڈالتا ۔۰


In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="bartowski/Tower-Babel_Babel-9B-Chat-GGUF",
	filename="Tower-Babel_Babel-9B-Chat-Q8_0.gguf",
)


ModuleNotFoundError: No module named 'llama_cpp'

In [ ]:
# Define a chat prompt in Urdu
messages = [
    {
        "role": "user",
        "content": "محبت کے بارے میں ایک شعر کہو"  # "Write a poem about love"
    }
]

# Generate a response
response = llm.create_chat_completion(
    messages=messages,
    max_tokens=100,  # Maximum number of tokens to generate
    temperature=0.7,  # Controls randomness
    top_p=0.95,  # Nucleus sampling parameter
)

# Print the generated response
print("Generated Urdu Text:")
print(response["choices"][0]["message"]["content"])

Llama.generate: 36 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    1732.26 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =   20162.29 ms /   100 runs   (  201.62 ms per token,     4.96 tokens per second)
llama_perf_context_print:       total time =   20324.08 ms /   101 tokens


Generated Urdu Text:
محبت ہے، جو دلوں میں نور آگیا  
دل کو چھو کر، ہر بے دار کو شاداب بناگیا  
جہان کی ہر چیز کو جو سیکھا ہے یہ نام  
محبت ہے، دنیا کی سب سے بڑی دو


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
  "Tower-Babel/Babel-83B-Chat",
  torch_dtype=torch.bfloat16,
  device_map=device
)
tokenizer = AutoTokenizer.from_pretrained("Tower-Babel/Babel-83B-Chat")

# prepare messages to model
prompt = "Hiii How are you?"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
print(f"Formatted text:\n {text}")
print(f"Model input:\n {model_inputs}")

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

print(f"Response:\n {response[0]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.9k [00:00<?, ?B/s]

model-00001-of-00035.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00002-of-00035.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

model-00033-of-00035.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

model-00034-of-00035.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00035-of-00035.safetensors:   0%|          | 0.00/2.49G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/35 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 462.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 211.38 MiB is free. Process 6486 has 21.95 GiB memory in use. Of the allocated memory 21.77 GiB is allocated by PyTorch, and 1.39 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
## Memory Error